# Supermarket Sales Data Pipeline

Complete ETL pipeline demonstration:
1. Extract data from Kaggle
2. Transform into dimensional model (2 dimensions + 1 fact table)
3. Load into SQLite database using external SQL schema
4. Generate automated reports with external SQL queries

## Setup

Import required libraries and set up the environment.

In [1]:
import sys
import os
sys.path.append('..')

import pandas as pd
import sqlite3
from pathlib import Path
from simple_pipeline import extract_data, transform_data, load_data, generate_reports

print("✅ Pipeline modules imported successfully")
print(f"📂 Current working directory: {os.getcwd()}")

✅ Pipeline modules imported successfully
📂 Current working directory: /app/notebooks


## Step 1: Extract Data from Kaggle

Download and load the supermarket sales dataset from Kaggle.

In [2]:
# Extract raw data from Kaggle
print("🚀 Starting data extraction...")
raw_df = extract_data()

if raw_df is not None:
    print(f"\n📊 Dataset Overview:")
    print(f"  Rows: {len(raw_df)}")
    print(f"  Columns: {len(raw_df.columns)}")
    print(f"  Column names: {list(raw_df.columns)}")
    
    print(f"\n📋 Sample data:")
    display(raw_df.head(3))
else:
    print("❌ Data extraction failed. Please check Kaggle credentials.")

🚀 Starting data extraction...
🔄 Extracting data from Kaggle...
Dataset URL: https://www.kaggle.com/datasets/lovishbansal123/sales-of-a-supermarket
✅ Extracted 1000 rows with 17 columns

📊 Dataset Overview:
  Rows: 1000
  Columns: 17
  Column names: ['Invoice ID', 'Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Unit price', 'Quantity', 'Tax 5%', 'Total', 'Date', 'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income', 'Rating']

📋 Sample data:


,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4


## Step 2: Transform Data into Dimensional Model

Transform raw transaction data into star schema with:
- **dim_product**: Product dimension table
- **dim_store**: Store dimension table  
- **fact_sales**: Sales fact table with foreign keys

In [3]:
# Transform data into dimensional model
print("🔄 Starting data transformation...")
dim_product, dim_store, fact_sales = transform_data(raw_df)

print("\n📦 Product Dimension (dim_product):")
print(f"  Shape: {dim_product.shape}")
display(dim_product.head())

print("\n🏪 Store Dimension (dim_store):") 
print(f"  Shape: {dim_store.shape}")
display(dim_store)

print("\n💰 Sales Fact Table (fact_sales):")
print(f"  Shape: {fact_sales.shape}")
print(f"  Columns: {list(fact_sales.columns)}")
display(fact_sales.head())

🔄 Starting data transformation...
🔄 Transforming data...
📋 Available columns: ['invoice_id', 'branch', 'city', 'customer_type', 'gender', 'product_line', 'unit_price', 'quantity', 'tax_5%', 'total', 'date', 'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income', 'rating']
📊 Data shape: (1000, 17)
📋 Fact columns after rename: ['invoice_id', 'branch', 'city', 'customer_type', 'gender', 'product_line', 'unit_price', 'quantity', 'tax_5', 'total', 'date', 'time', 'payment', 'cogs', 'gross_margin_percentage', 'gross_income', 'rating', 'product_id', 'store_id']
✅ Created: 998 products, 3 stores, 1000 sales transactions

📦 Product Dimension (dim_product):
  Shape: (998, 5)


,product_id,product_line,unit_price,cogs,gross_margin_percentage
0,1,Health and beauty,74.69,522.83,4.761905
1,2,Electronic accessories,15.28,76.40,4.761905
2,3,Home and lifestyle,46.33,324.31,4.761905
3,4,Health and beauty,58.22,465.76,4.761905
4,5,Sports and travel,86.31,604.17,4.761905



🏪 Store Dimension (dim_store):
  Shape: (3, 3)


,store_id,branch,city
0,1,A,Yangon
1,2,C,Naypyitaw
2,3,B,Mandalay



💰 Sales Fact Table (fact_sales):
  Shape: (1000, 14)
  Columns: ['sales_id', 'product_id', 'store_id', 'invoice_id', 'customer_type', 'gender', 'quantity', 'tax_5', 'total', 'date', 'time', 'payment', 'gross_income', 'rating']


,sales_id,product_id,store_id,invoice_id,customer_type,gender,quantity,tax_5,total,date,time,payment,gross_income,rating
0,1,1,1,750-67-8428,Member,Female,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,26.1415,9.1
1,2,2,2,226-31-3081,Normal,Female,5,3.8200,80.2200,3/8/2019,10:29,Cash,3.8200,9.6
2,3,3,1,631-41-3108,Normal,Male,7,16.2155,340.5255,3/3/2019,13:23,Credit card,16.2155,7.4
3,4,4,1,123-19-1176,Member,Male,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,23.2880,8.4
4,5,5,1,373-73-7910,Normal,Male,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,30.2085,5.3


## Step 3: Load Data into SQLite Database

Create database schema from external SQL file and load the dimensional data.

In [4]:
# Clean up any existing database for fresh start
db_path = "../data/supermarket_sales.db"
if os.path.exists(db_path):
    os.remove(db_path)
    print("🗑️ Removed existing database")

# Load data into SQLite database using external SQL schema
db_path = load_data(dim_product, dim_store, fact_sales)

# Verify the database was created correctly
with sqlite3.connect(db_path) as conn:
    print("\n📊 Database verification:")
    
    # Check table row counts
    for table in ['dim_product', 'dim_store', 'fact_sales']:
        count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
        print(f"  {table}: {count} rows")
    
    # Check actual schema
    fact_columns = pd.read_sql("PRAGMA table_info(fact_sales)", conn)
    print(f"\n📋 Fact table schema: {list(fact_columns['name'])}")
    
    # Sample data verification
    sample = pd.read_sql("SELECT * FROM fact_sales LIMIT 3", conn)
    print("\n📊 Sample fact_sales data:")
    display(sample)

🔄 Loading data into database...
✅ Database schema created from SQL file: /app/sql/create_tables.sql
✅ Data loaded successfully

📊 Database verification:
  dim_product: 998 rows
  dim_store: 3 rows
  fact_sales: 1000 rows

📋 Fact table schema: ['sales_id', 'product_id', 'store_id', 'invoice_id', 'customer_type', 'gender', 'quantity', 'tax_5', 'total', 'date', 'time', 'payment', 'gross_income', 'rating']

📊 Sample fact_sales data:


,sales_id,product_id,store_id,invoice_id,customer_type,gender,quantity,tax_5,total,date,time,payment,gross_income,rating
0,1,1,1,750-67-8428,Member,Female,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,26.1415,9.1
1,2,2,2,226-31-3081,Normal,Female,5,3.8200,80.2200,3/8/2019,10:29,Cash,3.8200,9.6
2,3,3,1,631-41-3108,Normal,Male,7,16.2155,340.5255,3/3/2019,13:23,Credit card,16.2155,7.4


## Step 4: Generate Automated Reports

Execute external SQL queries to generate business reports with joins and window functions.

In [5]:
# Generate automated reports using external SQL files
generate_reports(db_path)

🔄 Generating reports...
📋 Fact table columns: ['sales_id', 'product_id', 'store_id', 'invoice_id', 'customer_type', 'gender', 'quantity', 'tax_5', 'total', 'date', 'time', 'payment', 'gross_income', 'rating']

📊 SALES BY PRODUCT LINE
             product_line  transaction_count  total_revenue  \
0      Food and beverages                174       56144.84   
1       Sports and travel                166       55122.83   
2  Electronic accessories                170       54337.53   
3     Fashion accessories                178       54305.89   
4      Home and lifestyle                160       53861.91   
5       Health and beauty                152       49193.74   

   avg_transaction_value  avg_rating  
0                 322.67        7.11  
1                 332.07        6.92  
2                 319.63        6.92  
3                 305.09        7.03  
4                 336.64        6.84  
5                 323.64        7.00  

📊 SALES BY STORE
        city branch  transaction_

## Step 5: Custom Analysis

Run additional analysis using external SQL queries.

In [6]:
# Custom analysis: Product performance by city using external SQL
with sqlite3.connect(db_path) as conn:
    try:
        # Execute the product performance by city query
        with open('../sql/report_product_performance_by_city.sql', 'r') as f:
            query = f.read()
        
        result = pd.read_sql_query(query, conn)
        print("📊 Product Performance by City (from external SQL file):")
        display(result)
        
    except Exception as e:
        print(f"❌ Error executing custom analysis: {e}")
        
        # Debug information
        tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", conn)
        print(f"\n🔍 Available tables: {list(tables['name'])}")

📊 Product Performance by City (from external SQL file):


,product_line,city,branch,transaction_count,total_revenue,city_rank
0,Sports and travel,Mandalay,B,62,19988.20,1
1,Health and beauty,Mandalay,B,53,19980.66,2
2,Home and lifestyle,Mandalay,B,50,17549.16,3
3,Electronic accessories,Mandalay,B,55,17051.44,4
4,Fashion accessories,Mandalay,B,62,16413.32,5
5,Food and beverages,Mandalay,B,50,15214.89,6
6,Food and beverages,Naypyitaw,C,66,23766.85,1
7,Fashion accessories,Naypyitaw,C,65,21560.07,2
8,Electronic accessories,Naypyitaw,C,55,18968.97,3
9,Health and beauty,Naypyitaw,C,52,16615.33,4


## Pipeline Summary

The complete ETL pipeline:
1. ✅ **Extract**: Downloaded supermarket sales data from Kaggle API
2. ✅ **Transform**: Created dimensional model (star schema)
3. ✅ **Load**: Used external SQL schema to create database structure
4. ✅ **Report**: Generated business reports using external SQL queries

All SQL code is externalized into separate .sql files for maintainability.